In [1]:
import json
import pandas as pd
import plotly.express as px

In [2]:
# Read the json file into a DataFrame
with open('dataset/india_aqi.json', 'r') as file:
    data = json.load(file)
    
df = pd.DataFrame(data['records'])

# Print basic statistics and info about the DataFrame
df.describe(include='all').transpose()

,count,unique,top,freq
country,3381,1,India,3381
state,3381,30,Maharashtra,578
city,3381,274,Delhi,255
station,3381,510,"Lohiyanagar, Begusarai - BSPCB",7
last_update,3381,1,02-11-2025 00:00:00,3381
latitude,3381,510,25.42742023,7
longitude,3381,510,86.13886079,7
pollutant_id,3381,7,PM10,500
min_value,3381,199,NA,260
max_value,3381,361,NA,260


In [3]:
metadata = {
    key: value
    for key, value in data.items()
    if key
    not in [
        "records",
        "field",
        "target_bucket",
        "external_ws",
        "external_ws_url",
        "version",
        "status",
        "limit",
        "offset",
        "catalog_uuid",
        "visualizable",
        "active",
        "created",
        "updated",
        "message",
        "index_name"
    ]
}
pretty_json = json.dumps(metadata, indent=4)
print(pretty_json)

{
    "title": "Real time Air Quality Index from various locations",
    "desc": "Real time Air Quality Index from various locations",
    "org_type": "Central",
    "org": [
        "Ministry of Environment, Forest and Climate Change",
        "Central Pollution Control Board"
    ],
    "sector": [
        "Environment and Forest",
        "Industrial Air Pollution",
        "Residential Air Pollution",
        "Vehicular Air Pollution"
    ],
    "source": "data.gov.in",
    "created_date": "2016-04-08T00:00:00Z",
    "updated_date": "2025-11-01T19:19:33Z",
    "total": 3381,
    "count": 3381
}


In [4]:
# ----------------------------------------
# Clean data
# ----------------------------------------

# Replace common missing tokens
df.replace(["NA", "NaN", "None", ""], pd.NA, inplace=True)

# Convert to numeric safely
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")
df["avg_value"] = pd.to_numeric(df["avg_value"], errors="coerce")

# Drop rows with invalid coordinates or missing values
df = df.dropna(subset=["latitude", "longitude", "avg_value"])

# Optional: remove extreme values for better visualization
df = df[(df["avg_value"] > 0) & (df["avg_value"] < 1000)]

In [5]:
# ----------------------------------------
# Create interactive Plotly map
# ----------------------------------------
fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color="avg_value",
    size="avg_value",
    hover_name="city",
    hover_data={
        "state": True,
        "station": True,
        "pollutant_id": True,
        "avg_value": True,
        "latitude": False,
        "longitude": False,
    },
    color_continuous_scale="RdYlGn_r",
    title="🌏 Air Quality Monitoring Stations Across India",
    mapbox_style="carto-positron",
    zoom=4,
    height=700,
)

In [6]:
# ----------------------------------------
# Show map and export
# ----------------------------------------
fig.show()
fig.write_html("india_air_quality_map_plotly.html")

print("✅ Map generated successfully and saved as 'india_air_quality_map_plotly.html'")

✅ Map generated successfully and saved as 'india_air_quality_map_plotly.html'
